In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

test_df = pd.read_csv('test_updated.csv')
train_df = pd.read_csv('train_updated.csv')

target = train_df['RiskFlag']
train_ids = train_df['ProfileID']
test_ids = test_df['ProfileID']

train_features = train_df.drop(columns=['RiskFlag', 'ProfileID'])
test_features = test_df.drop(columns=['ProfileID'])

combined_df = pd.concat([train_features, test_features], axis=0)

# Encoding 
education_map = {"High School": 0, "Bachelor's": 1, "Master's": 2, "PhD": 3}
combined_df['QualificationLevel'] = combined_df['QualificationLevel'].map(education_map)

binary_map = {'Yes': 1, 'No': 0}
for col in ['OwnsProperty', 'FamilyObligation', 'JointApplicant']:
    combined_df[col] = combined_df[col].map(binary_map)


# Feature Engineering
# Financial ratios
combined_df['Loan_to_Income'] = combined_df['RequestedSum'] / combined_df['AnnualEarnings']
combined_df['EMI_Approx'] = combined_df['RequestedSum'] / combined_df['RepayPeriod']
combined_df['EMI_to_Income'] = combined_df['EMI_Approx'] / (combined_df['AnnualEarnings'] / 12)
combined_df['Debt_to_Loan'] = combined_df['DebtFactor'] / (combined_df['RequestedSum'] + 1e-6)
combined_df['Earnings_per_Account'] = combined_df['AnnualEarnings'] / (combined_df['ActiveAccounts'] + 1e-6)
combined_df['MonthlyIncome'] = combined_df['AnnualEarnings'] / 12
combined_df['Repayment_to_Earnings'] = combined_df['RequestedSum'] / (combined_df['AnnualEarnings'] * combined_df['RepayPeriod'] / 12 + 1e-6)
combined_df['Margin_after_Debt'] = combined_df['MonthlyIncome'] - (combined_df['DebtFactor'] * combined_df['MonthlyIncome'])

# Interaction features
combined_df['Trust_Qualification'] = combined_df['TrustMetric'] * combined_df['QualificationLevel']
combined_df['WorkDuration_Qualification'] = combined_df['WorkDuration'] * combined_df['QualificationLevel']

combined_df['Log_WorkDuration'] = np.log1p(combined_df['WorkDuration'])
combined_df['Log_ActiveAccounts'] = np.log1p(combined_df['ActiveAccounts'])

combined_df['WorkDuration_Bin'] = pd.cut(combined_df['WorkDuration'], bins=[-np.inf, 12, 60, np.inf], labels=[0,1,2])

# Age group bins: 18-30 young, 31-50 mid, 50+ senior
combined_df['Age_Group'] = pd.cut(combined_df['ApplicantYears'], bins=[0,30,50,np.inf], labels=[0,1,2])

# Property and JointApplicant interaction
combined_df['Property_JointApplicant'] = combined_df['OwnsProperty'] * combined_df['JointApplicant']

# Family obligation and WorkCategory combined flag for self-employed with family responsibilities
if 'WorkCategory_self-employed' in combined_df.columns:
    combined_df['SelfEmp_FamilyObligation'] = combined_df['WorkCategory_self-employed'] * combined_df['FamilyObligation']
else:
    combined_df['SelfEmp_FamilyObligation'] = 0

combined_df = pd.get_dummies(combined_df, columns=['WorkDuration_Bin', 'Age_Group'], drop_first=True)

for col in ['AnnualEarnings', 'RequestedSum']:
    combined_df[col] = np.log1p(combined_df[col])


combined_df = pd.get_dummies(combined_df, columns=['WorkCategory', 'RelationshipStatus', 'FundUseCase'], drop_first=True)

# Scaling
numeric_cols = combined_df.select_dtypes(include=['int64', 'float64']).columns
scaler = StandardScaler()
combined_df[numeric_cols] = scaler.fit_transform(combined_df[numeric_cols])



X = combined_df[:len(train_df)]
X_test_submission = combined_df[len(train_df):]
y = target

In [3]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_val)
print("Classification Report:\n", classification_report(y_val, y_pred))
print("Accuracy:", accuracy_score(y_val, y_pred))

test_preds = model.predict(X_test_submission)

submission_df = pd.DataFrame({
    'ProfileID': test_ids,
    'RiskFlag': test_preds
})

submission_df.to_csv('nv_submission.csv', index=False)

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.90      0.90     36073
           1       0.27      0.28      0.28      4783

    accuracy                           0.83     40856
   macro avg       0.59      0.59      0.59     40856
weighted avg       0.83      0.83      0.83     40856

Accuracy: 0.8273448208341492


In [4]:
import joblib
joblib.dump(model, "gaussian_nb.pkl")

['gaussian_nb.pkl']

In [5]:
import joblib
from sklearn.metrics import accuracy_score, classification_report

model = joblib.load("gaussian_nb.pkl")   

y_pred = model.predict(X_val)

print("Accuracy:", accuracy_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.8273448208341492

Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.90      0.90     36073
           1       0.27      0.28      0.28      4783

    accuracy                           0.83     40856
   macro avg       0.59      0.59      0.59     40856
weighted avg       0.83      0.83      0.83     40856

